In [9]:
import numpy as np           #linear algebra
import pandas as pd          #data processing, CSV File
import matplotlib.pyplot as plt     #
import seaborn as sns

In [10]:
cd Final Project

[Error 2] The system cannot find the file specified: u'Final Project'
C:\Users\fjmurias.DGS\Personal\Data Science\Final Project


In [11]:
# ALL YEARS DATA
df_regular_compact = pd.read_csv('RegularSeasonCompactResults.csv')
df_teams = pd.read_csv('Teams.csv')
df_seeds2018 = pd.read_csv('2018 TourneySeeds.csv')

In [12]:
# 2018 Specific Data
df_2018detailedregular = pd.read_csv('2018 RegularSeasonDetailedResults.csv')
df_seeds2018 = pd.read_csv('2018 TourneySeeds.csv')

In [13]:
df_2018detailedregular.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2018,11,1104,82,1272,70,N,0,26,57,...,17,22,36,19,24,7,17,7,3,22
1,2018,11,1107,69,1233,67,H,0,24,62,...,22,11,14,7,20,14,7,6,3,21
2,2018,11,1112,101,1319,67,H,0,34,57,...,13,17,30,12,11,11,13,3,1,24
3,2018,11,1113,94,1226,74,H,0,34,65,...,24,13,22,8,21,17,14,6,2,20
4,2018,11,1116,95,1359,56,H,0,36,63,...,29,8,15,15,17,12,19,6,1,20


In [14]:
def seed_to_int(seed):
    """Get just the digits from the seeding. Return as int"""
    s_int = int(seed[1:3])
    return s_int
df_seeds2018['Seed'] = df_seeds2018.Seed.apply(seed_to_int)
# df_seeds2017.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label

In [15]:
df_seeds2018.head()

,Season,Seed,TeamID
0,2018,1,1437
1,2018,2,1345
2,2018,3,1403
3,2018,4,1455
4,2018,5,1452


In [16]:
# 1. This merges teams with seeds
teamseeds2018 = pd.merge(left=df_teams, right=df_seeds2018, how='left', left_on='Team_Id',right_on='TeamID')
teamseeds2018['Name']=teamseeds2018['Team_Name']
teamseeds2018.head()

,Team_Id,Team_Name,Season,Seed,TeamID,Name
0,1101,Abilene Chr,NaN,NaN,NaN,Abilene Chr
1,1102,Air Force,NaN,NaN,NaN,Air Force
2,1103,Akron,NaN,NaN,NaN,Akron
3,1104,Alabama,2018.0,9.0,1104.0,Alabama
4,1105,Alabama A&M,NaN,NaN,NaN,Alabama A&M


In [17]:
teamseeds2018.Seed.fillna(value=' NA', inplace=True)
teamseeds2018.Season.fillna(value='NA', inplace=True)
teamseeds2018.TeamID.fillna(value='NA', inplace=True)
teamseeds2018.head()

,Team_Id,Team_Name,Season,Seed,TeamID,Name
0,1101,Abilene Chr,NA,NA,NA,Abilene Chr
1,1102,Air Force,NA,NA,NA,Air Force
2,1103,Akron,NA,NA,NA,Akron
3,1104,Alabama,2018,9,1104,Alabama
4,1105,Alabama A&M,NA,NA,NA,Alabama A&M


In [18]:
# 2a. This adds margin of victory as a column to df_2017compactregular
margin_of_victory = df_2018detailedregular.WScore - df_2018detailedregular.LScore
df_2018detailedregular['margin_of_victory'] = margin_of_victory

In [19]:
# 2b. This adds wins as a column to df_2017compactregular 
Wteam_wins = df_2018detailedregular.groupby('WTeamID')['WTeamID'].transform('count')
df_2018detailedregular['Wteam_wins'] = df_2018detailedregular.groupby('WTeamID')['WTeamID'].transform('count')
df_2018detailedregular['Lteam_losses'] = df_2018detailedregular.groupby('LTeamID')['LTeamID'].transform('count')
type(Wteam_wins)
Wteam_wins.head()

0    19
1    21
2    27
3    20
4    23
Name: WTeamID, dtype: int64

In [20]:
# 3. This will give you compact game stats merged with the above teams / seeds variable. Need to do this 
# 3. cont'd. with my current code because I can't merged a "groupby" column with a dataframe
# DGS = Detailed Game Stats
DGS2018 = pd.merge(left=df_2018detailedregular,right=df_teams, how='left', left_on='WTeamID', right_on='Team_Id') 
DGS2018.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LAst,LTO,LStl,LBlk,LPF,margin_of_victory,Wteam_wins,Lteam_losses,Team_Id,Team_Name
0,2018,11,1104,82,1272,70,N,0,26,57,...,7,17,7,3,22,12,19,13,1104,Alabama
1,2018,11,1107,69,1233,67,H,0,24,62,...,14,7,6,3,21,2,21,13,1107,Albany NY
2,2018,11,1112,101,1319,67,H,0,34,57,...,11,13,3,1,24,34,27,26,1112,Arizona
3,2018,11,1113,94,1226,74,H,0,34,65,...,17,14,6,2,20,20,20,16,1113,Arizona St
4,2018,11,1116,95,1359,56,H,0,36,63,...,12,19,6,1,20,39,23,22,1116,Arkansas


In [21]:
df_Wteams2018 = pd.DataFrame()
df_Wteams2018['Teams'] = DGS2018['WTeamID']
df_Wteams2018['Wins'] = DGS2018.groupby('WTeamID')['WTeamID'].transform('count')
df_Wteams2018['WPS'] = DGS2018.groupby('WTeamID')['WScore'].transform('sum').round(1)
df_Wteams2018['WOPS'] = DGS2018.groupby('WTeamID')['LScore'].transform('sum').round(1)
df_Wteams2018['WMOV'] = DGS2018.groupby('WTeamID')['margin_of_victory'].transform('sum').round(1)
df_Wteams2018['Wfgm'] = DGS2018.groupby('WTeamID')['WFGM'].transform('sum').round(1)
df_Wteams2018['W-Lfgm'] = DGS2018.groupby('WTeamID')['LFGM'].transform('sum').round(1)
df_Wteams2018['Wfga'] = DGS2018.groupby('WTeamID')['WFGA'].transform('sum').round(1)
df_Wteams2018['W-Lfga'] = DGS2018.groupby('WTeamID')['LFGA'].transform('sum').round(1)
df_Wteams2018['Wfgm3'] = DGS2018.groupby('WTeamID')['WFGM3'].transform('sum').round(1)
df_Wteams2018['W-Lfgm3'] = DGS2018.groupby('WTeamID')['LFGM3'].transform('sum').round(1)
df_Wteams2018['Wfga3'] = DGS2018.groupby('WTeamID')['WFGA3'].transform('sum').round(1)
df_Wteams2018['W-Lfga3'] = DGS2018.groupby('WTeamID')['LFGA3'].transform('sum').round(1)
df_Wteams2018['Wftm'] = DGS2018.groupby('WTeamID')['WFTM'].transform('sum').round(1)
df_Wteams2018['W-Lftm'] = DGS2018.groupby('WTeamID')['LFTM'].transform('sum').round(1)
df_Wteams2018['Wfta'] = DGS2018.groupby('WTeamID')['WFTA'].transform('sum').round(1)
df_Wteams2018['W-Lfta'] = DGS2018.groupby('WTeamID')['LFTA'].transform('sum').round(1)
df_Wteams2018['WOR'] = DGS2018.groupby('WTeamID')['WOR'].transform('sum').round(1)
df_Wteams2018['W-LOR'] = DGS2018.groupby('WTeamID')['LOR'].transform('sum').round(1)
df_Wteams2018['WORP'] = DGS2018.groupby('WTeamID')['WOR'].transform('sum') + DGS2018.groupby('WTeamID')['LDR'].transform('sum')
df_Wteams2018['WDR'] = DGS2018.groupby('WTeamID')['WDR'].transform('sum').round(1)
df_Wteams2018['W-LDR'] = DGS2018.groupby('WTeamID')['LDR'].transform('sum').round(1)
df_Wteams2018['WDRP'] = DGS2018.groupby('WTeamID')['WDR'].transform('sum') + DGS2018.groupby('WTeamID')['LOR'].transform('sum')
df_Wteams2018['WAST'] = DGS2018.groupby('WTeamID')['WAst'].transform('sum').round(1)
df_Wteams2018['WTO'] = DGS2018.groupby('WTeamID')['WTO'].transform('sum').round(1)
df_Wteams2018['W-LTO'] = DGS2018.groupby('WTeamID')['LTO'].transform('sum').round(1)
df_Wteams2018['WSTL'] = DGS2018.groupby('WTeamID')['WStl'].transform('sum').round(1)
df_Wteams2018['WBLK'] = DGS2018.groupby('WTeamID')['WBlk'].transform('sum').round(1)
df_Wteams2018['WPF'] = DGS2018.groupby('WTeamID')['WPF'].transform('sum').round(1)
df_Wteams2018.sort_values(by = 'Teams', ascending=True, inplace = True)
df_Wteams2018.drop_duplicates(inplace=True)
df_Wteams2018.shape

(351, 29)

In [22]:
df_Wteams2018.head()

,Teams,Wins,WPS,WOPS,WMOV,Wfgm,W-Lfgm,Wfga,W-Lfga,Wfgm3,...,WORP,WDR,W-LDR,WDRP,WAST,WTO,W-LTO,WSTL,WBLK,WPF
3608,1101,12,915,827,88,329,286,687,672,95,...,398,307,293,426,198,149,169,75,57,239
2749,1102,10,766,675,91,247,239,567,559,96,...,350,249,247,348,157,111,145,62,30,184
3260,1103,13,1005,915,90,346,317,728,720,136,...,424,333,300,448,198,178,169,75,34,255
2094,1104,19,1476,1242,234,530,435,1047,1154,130,...,611,556,425,786,265,289,262,129,116,350
2226,1105,3,202,178,24,67,71,140,208,19,...,83,87,52,140,40,60,36,16,8,46


In [23]:
df_Lteams2018 = pd.DataFrame()
df_Lteams2018['Teams'] = DGS2018['LTeamID']
df_Lteams2018['Losses'] = DGS2018.groupby('LTeamID')['LTeamID'].transform('count')
df_Lteams2018['LPS'] = DGS2018.groupby('LTeamID')['LScore'].transform('sum').round(1)
df_Lteams2018['LOPS'] = DGS2018.groupby('LTeamID')['WScore'].transform('sum').round(1)
df_Lteams2018['LMOV'] = DGS2018.groupby('LTeamID')['margin_of_victory'].transform('sum').round(1)
df_Lteams2018['Lfgm'] = DGS2018.groupby('LTeamID')['LFGM'].transform('sum').round(1)
df_Lteams2018['L-Wfgm'] = DGS2018.groupby('LTeamID')['WFGM'].transform('sum').round(1)
df_Lteams2018['Lfga'] = DGS2018.groupby('LTeamID')['LFGA'].transform('sum').round(1)
df_Lteams2018['L-Wfga'] = DGS2018.groupby('LTeamID')['WFGA'].transform('sum').round(1)
df_Lteams2018['Lfgm3'] = DGS2018.groupby('LTeamID')['LFGM3'].transform('sum').round(1)
df_Lteams2018['L-Wfgm3'] = DGS2018.groupby('LTeamID')['WFGM3'].transform('sum').round(1)
df_Lteams2018['Lfga3'] = DGS2018.groupby('LTeamID')['LFGA3'].transform('sum').round(1)
df_Lteams2018['L-Wfga3'] = DGS2018.groupby('LTeamID')['WFGA3'].transform('sum').round(1)
df_Lteams2018['Lftm'] = DGS2018.groupby('LTeamID')['LFTM'].transform('sum').round(1)
df_Lteams2018['L-Wftm'] = DGS2018.groupby('LTeamID')['WFTM'].transform('sum').round(1)
df_Lteams2018['Lfta'] = DGS2018.groupby('LTeamID')['LFTA'].transform('sum').round(1)
df_Lteams2018['L-Wfta'] = DGS2018.groupby('LTeamID')['WFTA'].transform('sum').round(1)
df_Lteams2018['LOR'] = DGS2018.groupby('LTeamID')['LOR'].transform('sum').round(1)
df_Lteams2018['L-WOR'] = DGS2018.groupby('LTeamID')['WOR'].transform('sum').round(1)
df_Lteams2018['LORP'] = DGS2018.groupby('LTeamID')['LOR'].transform('sum') + DGS2018.groupby('LTeamID')['WDR'].transform('sum')
df_Lteams2018['LDR'] = DGS2018.groupby('LTeamID')['LDR'].transform('sum').round(1)
df_Lteams2018['L-WDR'] = DGS2018.groupby('LTeamID')['WDR'].transform('sum').round(1)
df_Lteams2018['LDRP'] = DGS2018.groupby('LTeamID')['LDR'].transform('sum') + DGS2018.groupby('LTeamID')['WOR'].transform('sum')
df_Lteams2018['LAST'] = DGS2018.groupby('LTeamID')['LAst'].transform('sum').round(1)
df_Lteams2018['LTO'] = DGS2018.groupby('LTeamID')['LTO'].transform('sum').round(1)
df_Lteams2018['L-WTO'] = DGS2018.groupby('LTeamID')['WTO'].transform('sum').round(1)
df_Lteams2018['LSTL'] = DGS2018.groupby('LTeamID')['LStl'].transform('sum').round(1)
df_Lteams2018['LBLK'] = DGS2018.groupby('LTeamID')['LBlk'].transform('sum').round(1)
df_Lteams2018['LPF'] = DGS2018.groupby('LTeamID')['LPF'].transform('sum').round(1)
df_Lteams2018.sort_values(by = 'Teams', ascending=True, inplace = True)
df_Lteams2018.drop_duplicates(inplace=True)
df_Lteams2018.shape

(351, 29)

In [24]:
TD = pd.DataFrame()
TD = pd.merge(left=df_Wteams2018,right=df_Lteams2018, how='left', left_on='Teams', right_on='Teams') 
TD.sort_values(by = 'Teams', ascending=True, inplace = True)
TD.head()

,Teams,Wins,WPS,WOPS,WMOV,Wfgm,W-Lfgm,Wfga,W-Lfga,Wfgm3,...,LORP,LDR,L-WDR,LDRP,LAST,LTO,L-WTO,LSTL,LBLK,LPF
0,1101,12,915,827,88,329,286,687,672,95,...,522,320,383,469,177,240,249,118,41,329
1,1102,10,766,675,91,247,239,567,559,96,...,690,409,508,576,246,259,260,125,46,342
2,1103,13,1005,915,90,346,317,728,720,136,...,661,395,488,569,203,251,234,110,40,366
3,1104,19,1476,1242,234,530,435,1047,1154,130,...,563,337,408,501,161,192,186,89,70,296
4,1105,3,202,178,24,67,71,140,208,19,...,1035,624,731,914,300,454,317,107,41,460


In [25]:
# Four Factors Model - Shooting 40%, Turnovers 25%, Rebounding 20%, Free Throws 15%
# Effective Field Goal Percentage (eFG%) or (FG + 0.5 * 3P) / FGA. need for offense and defense
# Turnover Rate=Turnovers/(Field Goal Attempts + 0.44*Free Throw Attempts + Turnovers)
# Offensive Rebounding Percentage = (Offensive Rebounds)/[(Offensive Rebounds)+(Opponent’s Defensive Rebounds)] 
# defensive rebounding DRB / (Opp ORB + DRB)
# free throws is FT / FGA

In [26]:
TD['OeFG%'] = ((TD['Wfgm'] + TD['Lfgm'])+0.5*(TD['Wfgm3'] + TD['Lfgm3']))/(TD['Wfga'] + TD['Lfga'])
TD['DeFG%'] = ((TD['W-Lfgm'] + TD['L-Wfgm'])+0.5*(TD['W-Lfgm3'] + TD['L-Wfgm3']))/(TD['W-Lfga'] + TD['L-Wfga'])
TD['OeFG%'] = TD['OeFG%'].round(2)
TD['DeFG%'] = TD['DeFG%'].round(2)

In [27]:
TD['OTOV%'] = (TD['WTO']+TD['LTO'])/((TD['Wfga'] + TD['Lfga'])+0.44*(TD['Wfta'] + TD['Lfta'])+(TD['WTO']+TD['LTO'])).round(1)
TD['DTOV%'] = (TD['W-LTO']+TD['L-WTO'])/((TD['W-Lfga'] + TD['L-Wfga'])+0.44*(TD['W-Lfta'] + TD['L-Wfta'])+(TD['W-LTO']+TD['L-WTO']))
TD['OTOV%'] = TD['OTOV%'].round(2)
TD['DTOV%'] = TD['DTOV%'].round(2)

In [28]:
TD['ORB'] = (TD['WOR']+TD['LOR'])/((TD['WOR']+TD['W-LDR'])+(TD['LOR']+TD['L-WDR']))
TD['DRB'] = (TD['WDR']+TD['LDR'])/((TD['WDR']+TD['W-LOR'])+(TD['LDR']+TD['L-WOR']))
TD['ORB'] = TD['ORB'].round(2)
TD['DRB'] = TD['DRB'].round(2)

In [29]:
TD['FTP'] = (TD['Wftm']+TD['Lftm'])/(TD['Wfta']+TD['Lfta'])
TD['FTP'] = TD['FTP'].round(2)

In [30]:
TD['Games'] = TD['Wins'] + TD['Losses'] 
TD['WIN'] = TD['Wins'] / TD['Games']
TD['WIN'] = TD['WIN'].round(2)
TD['PPG'] = ((TD['WPS'] + TD['LPS'])/TD['Games']).round(1)
TD['OPPG'] = ((TD['WOPS'] + TD['LOPS'])/TD['Games']).round(1)
TD['MOV'] = TD['PPG'] - TD['OPPG']
TD.head()

,Teams,Wins,WPS,WOPS,WMOV,Wfgm,W-Lfgm,Wfga,W-Lfga,Wfgm3,...,OTOV%,DTOV%,ORB,DRB,FTP,Games,WIN,PPG,OPPG,MOV
0,1101,12,915,827,88,329,286,687,672,95,...,0.18,0.19,0.27,0.70,0.70,27,0.44,70.7,72.8,-2.1
1,1102,10,766,675,91,247,239,567,559,96,...,0.17,0.18,0.27,0.71,0.74,29,0.34,66.7,72.7,-6.0
2,1103,13,1005,915,90,346,317,728,720,136,...,0.18,0.17,0.27,0.72,0.70,31,0.42,71.3,76.2,-4.9
3,1104,19,1476,1242,234,530,435,1047,1154,130,...,0.18,0.16,0.29,0.69,0.67,34,0.56,72.4,70.0,2.4
4,1105,3,202,178,24,67,71,140,208,19,...,0.21,0.14,0.30,0.67,0.65,31,0.10,60.4,76.4,-16.0


In [31]:
TD = TD.reindex_axis(['Teams','Wins','Losses','WIN','OeFG%','OTOV%','ORB','DeFG%','DTOV%','DRB','FTP','PPG','OPPG','MOV'], axis=1)

In [32]:
TD.head()

,Teams,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
0,1101,12,15,0.44,0.51,0.18,0.27,0.52,0.19,0.70,0.70,70.7,72.8,-2.1
1,1102,10,19,0.34,0.48,0.17,0.27,0.55,0.18,0.71,0.74,66.7,72.7,-6.0
2,1103,13,18,0.42,0.52,0.18,0.27,0.54,0.17,0.72,0.70,71.3,76.2,-4.9
3,1104,19,15,0.56,0.52,0.18,0.29,0.47,0.16,0.69,0.67,72.4,70.0,2.4
4,1105,3,28,0.10,0.45,0.21,0.30,0.54,0.14,0.67,0.65,60.4,76.4,-16.0


In [33]:
teaminfo2018 = pd.DataFrame()
teaminfo2018 = pd.merge(left=TD,right=teamseeds2018, how='left', left_on='Teams', right_on='Team_Id') 
teaminfo2018 = teaminfo2018.reindex_axis(['Teams','Name','Seed','Wins','Losses','WIN','OeFG%','OTOV%','ORB','DeFG%','DTOV%','DRB','FTP','PPG','OPPG','MOV'], axis=1)
teaminfo2018.head()

,Teams,Name,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
0,1101,Abilene Chr,NA,12,15,0.44,0.51,0.18,0.27,0.52,0.19,0.70,0.70,70.7,72.8,-2.1
1,1102,Air Force,NA,10,19,0.34,0.48,0.17,0.27,0.55,0.18,0.71,0.74,66.7,72.7,-6.0
2,1103,Akron,NA,13,18,0.42,0.52,0.18,0.27,0.54,0.17,0.72,0.70,71.3,76.2,-4.9
3,1104,Alabama,9,19,15,0.56,0.52,0.18,0.29,0.47,0.16,0.69,0.67,72.4,70.0,2.4
4,1105,Alabama A&M,NA,3,28,0.10,0.45,0.21,0.30,0.54,0.14,0.67,0.65,60.4,76.4,-16.0


In [34]:
Win_FF = pd.DataFrame()
Win_FF = pd.merge(left=df_2018detailedregular,right=teaminfo2018, how='left', left_on='WTeamID', right_on='Teams') 
Win_FF.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
0,2018,11,1104,82,1272,70,N,0,26,57,...,0.52,0.18,0.29,0.47,0.16,0.69,0.67,72.4,70.0,2.4
1,2018,11,1107,69,1233,67,H,0,24,62,...,0.50,0.15,0.35,0.51,0.15,0.76,0.75,73.3,67.9,5.4
2,2018,11,1112,101,1319,67,H,0,34,57,...,0.57,0.16,0.33,0.49,0.15,0.75,0.76,80.9,71.2,9.7
3,2018,11,1113,94,1226,74,H,0,34,65,...,0.54,0.13,0.28,0.51,0.18,0.69,0.74,83.5,75.3,8.2
4,2018,11,1116,95,1359,56,H,0,36,63,...,0.54,0.13,0.29,0.50,0.17,0.68,0.68,81.1,75.5,5.6


In [35]:
Win_FF = Win_FF.reindex_axis(['WTeamID','LTeamID','WLoc','WScore','LScore','Name','Seed','Wins','Losses','WIN','OeFG%','OTOV%','ORB','DeFG%','DTOV%','DRB','FTP','PPG','OPPG','MOV'], axis=1)
Win_FF.shape

(5405, 20)

In [36]:
Win_FF.rename(columns={'Wteam': 'W_Team','Lteam': 'L_Team','WLoc': 'W_loc','Name': 'W_Name', 'Seed': 'W_Seed','Wins': 'W_Wins','Losses': 'W_Losses','WIN': 'W_WIN','OeFG%': 'W_OeFG%','OTOV%': 'W_OTOV%','ORB': 'W_ORB','DeFG%': 'W_DeFG%','DTOV%': 'W_DTOV%','DRB': 'W_DRB','FTP': 'W_FTP','PPG': 'W_PPG','OPPG': 'W_OPPG','MOV':'W_MOV'}, inplace=True)

In [37]:
Win_FF.head()

,WTeamID,LTeamID,W_loc,WScore,LScore,W_Name,W_Seed,W_Wins,W_Losses,W_WIN,W_OeFG%,W_OTOV%,W_ORB,W_DeFG%,W_DTOV%,W_DRB,W_FTP,W_PPG,W_OPPG,W_MOV
0,1104,1272,N,82,70,Alabama,9,19,15,0.56,0.52,0.18,0.29,0.47,0.16,0.69,0.67,72.4,70.0,2.4
1,1107,1233,H,69,67,Albany NY,NA,21,10,0.68,0.50,0.15,0.35,0.51,0.15,0.76,0.75,73.3,67.9,5.4
2,1112,1319,H,101,67,Arizona,4,27,7,0.79,0.57,0.16,0.33,0.49,0.15,0.75,0.76,80.9,71.2,9.7
3,1113,1226,H,94,74,Arizona St,11,20,11,0.65,0.54,0.13,0.28,0.51,0.18,0.69,0.74,83.5,75.3,8.2
4,1116,1359,H,95,56,Arkansas,7,23,11,0.68,0.54,0.13,0.29,0.50,0.17,0.68,0.68,81.1,75.5,5.6


In [38]:
kenpom2018 = pd.read_csv('2018 Kenpom.csv')
kenpom2018.rename(columns={'OrdinalRank': 'Rank'}, inplace=True)
kenpom2018.head()

,Season,RankingDayNum,SystemName,TeamID,Rank
0,2018,133,POM,1101,265
1,2018,133,POM,1102,232
2,2018,133,POM,1103,258
3,2018,133,POM,1104,51
4,2018,133,POM,1105,351


In [39]:
Win_FFKP = pd.DataFrame()
Win_FFKP = pd.merge(left=Win_FF,right=kenpom2018, how='left', left_on='WTeamID', right_on='TeamID') 
Win_FFKP.head()

,WTeamID,LTeamID,W_loc,WScore,LScore,W_Name,W_Seed,W_Wins,W_Losses,W_WIN,...,W_DRB,W_FTP,W_PPG,W_OPPG,W_MOV,Season,RankingDayNum,SystemName,TeamID,Rank
0,1104,1272,N,82,70,Alabama,9,19,15,0.56,...,0.69,0.67,72.4,70.0,2.4,2018,133,POM,1104,51
1,1107,1233,H,69,67,Albany NY,NA,21,10,0.68,...,0.76,0.75,73.3,67.9,5.4,2018,133,POM,1107,152
2,1112,1319,H,101,67,Arizona,4,27,7,0.79,...,0.75,0.76,80.9,71.2,9.7,2018,133,POM,1112,21
3,1113,1226,H,94,74,Arizona St,11,20,11,0.65,...,0.69,0.74,83.5,75.3,8.2,2018,133,POM,1113,45
4,1116,1359,H,95,56,Arkansas,7,23,11,0.68,...,0.68,0.68,81.1,75.5,5.6,2018,133,POM,1116,37


In [40]:
Win_FFKP.rename(columns={'Rank': 'W_Rank'}, inplace=True)

In [41]:
Win_FFKP.head()

,WTeamID,LTeamID,W_loc,WScore,LScore,W_Name,W_Seed,W_Wins,W_Losses,W_WIN,...,W_DRB,W_FTP,W_PPG,W_OPPG,W_MOV,Season,RankingDayNum,SystemName,TeamID,W_Rank
0,1104,1272,N,82,70,Alabama,9,19,15,0.56,...,0.69,0.67,72.4,70.0,2.4,2018,133,POM,1104,51
1,1107,1233,H,69,67,Albany NY,NA,21,10,0.68,...,0.76,0.75,73.3,67.9,5.4,2018,133,POM,1107,152
2,1112,1319,H,101,67,Arizona,4,27,7,0.79,...,0.75,0.76,80.9,71.2,9.7,2018,133,POM,1112,21
3,1113,1226,H,94,74,Arizona St,11,20,11,0.65,...,0.69,0.74,83.5,75.3,8.2,2018,133,POM,1113,45
4,1116,1359,H,95,56,Arkansas,7,23,11,0.68,...,0.68,0.68,81.1,75.5,5.6,2018,133,POM,1116,37


In [42]:
Combined_FFPKP = pd.DataFrame()
Combined_FFPKP = pd.merge(left=Win_FFKP,right=teaminfo2018, how='left', left_on='LTeamID', right_on='Teams') 
Combined_FFPKP.rename(columns={'Name': 'L_Name', 'Seed': 'L_Seed','Wins': 'L_Wins','Losses': 'L_Losses','WIN': 'L_WIN','OeFG%': 'L_OeFG%','OTOV%': 'L_OTOV%','ORB': 'L_ORB','DeFG%': 'L_DeFG%','DTOV%': 'L_DTOV%','DRB': 'L_DRB','FTP': 'L_FTP','PPG': 'L_PPG','OPPG': 'L_OPPG','MOV': 'L_MOV'}, inplace=True)
Combined_FFPKP.shape

(5405, 41)

In [43]:
Combined_FFPKP.head()

,WTeamID,LTeamID,W_loc,WScore,LScore,W_Name,W_Seed,W_Wins,W_Losses,W_WIN,...,L_OeFG%,L_OTOV%,L_ORB,L_DeFG%,L_DTOV%,L_DRB,L_FTP,L_PPG,L_OPPG,L_MOV
0,1104,1272,N,82,70,Alabama,9,19,15,0.56,...,0.51,0.18,0.35,0.51,0.16,0.66,0.69,70.8,71.2,-0.4
1,1107,1233,H,69,67,Albany NY,NA,21,10,0.68,...,0.54,0.14,0.24,0.52,0.16,0.70,0.74,79.8,76.2,3.6
2,1112,1319,H,101,67,Arizona,4,27,7,0.79,...,0.45,0.19,0.27,0.55,0.17,0.70,0.70,65.0,78.7,-13.7
3,1113,1226,H,94,74,Arizona St,11,20,11,0.65,...,0.55,0.17,0.20,0.53,0.15,0.72,0.68,73.1,75.1,-2.0
4,1116,1359,H,95,56,Arkansas,7,23,11,0.68,...,0.54,0.17,0.25,0.58,0.16,0.65,0.77,76.2,83.9,-7.7


In [44]:
Combined_FF = pd.DataFrame()
Combined_FF = pd.merge(left=Combined_FFPKP,right=kenpom2018, how='left', left_on='LTeamID', right_on='TeamID') 
Combined_FF.head()
#Combined_FF.shape

,WTeamID,LTeamID,W_loc,WScore,LScore,W_Name,W_Seed,W_Wins,W_Losses,W_WIN,...,L_DRB,L_FTP,L_PPG,L_OPPG,L_MOV,Season_y,RankingDayNum_y,SystemName_y,TeamID_y,Rank
0,1104,1272,N,82,70,Alabama,9,19,15,0.56,...,0.66,0.69,70.8,71.2,-0.4,2018,133,POM,1272,161
1,1107,1233,H,69,67,Albany NY,NA,21,10,0.68,...,0.70,0.74,79.8,76.2,3.6,2018,133,POM,1233,134
2,1112,1319,H,101,67,Arizona,4,27,7,0.79,...,0.70,0.70,65.0,78.7,-13.7,2018,133,POM,1319,326
3,1113,1226,H,94,74,Arizona St,11,20,11,0.65,...,0.72,0.68,73.1,75.1,-2.0,2018,133,POM,1226,221
4,1116,1359,H,95,56,Arkansas,7,23,11,0.68,...,0.65,0.77,76.2,83.9,-7.7,2018,133,POM,1359,297


In [45]:
Combined_FF.rename(columns={'Rank': 'L_Rank'}, inplace=True)

In [46]:
Combined_FF['Home_Team'] = [row['WTeamID'] if row['W_loc'] == 'H' else row['LTeamID'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_Rank'] = [row['W_Rank'] if row['W_loc'] == 'H' else row['L_Rank'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_Score'] = [row['WScore'] if row['W_loc'] == 'H' else row['LScore'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_Name'] = [row['W_Name'] if row['W_loc'] == 'H' else row['L_Name'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_Seed'] = [row['W_Seed'] if row['W_loc'] == 'H' else row['L_Seed'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_Wins'] = [row['W_Wins'] if row['W_loc'] == 'H' else row['L_Wins'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_Losses'] = [row['W_Losses'] if row['W_loc'] == 'H' else row['L_Losses'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_WIN%'] = [row['W_WIN'] if row['W_loc'] == 'H' else row['L_WIN'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_OeFG%'] = [row['W_OeFG%'] if row['W_loc'] == 'H' else row['L_OeFG%'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_OTOV%'] = [row['W_OTOV%'] if row['W_loc'] == 'H' else row['L_OTOV%'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_ORB'] = [row['W_ORB'] if row['W_loc'] == 'H' else row['L_ORB'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_DeFG%'] = [row['W_DeFG%'] if row['W_loc'] == 'H' else row['L_DeFG%'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_DTOV%'] = [row['W_DTOV%'] if row['W_loc'] == 'H' else row['L_DTOV%'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_DRB'] = [row['W_DRB'] if row['W_loc'] == 'H' else row['L_DRB'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_FTP'] = [row['W_FTP'] if row['W_loc'] == 'H' else row['L_FTP'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_PPG'] = [row['W_PPG'] if row['W_loc'] == 'H' else row['L_PPG'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_OPPG'] = [row['W_OPPG'] if row['W_loc'] == 'H' else row['L_OPPG'] for i, row in Combined_FF.iterrows()]
Combined_FF['H_MOV'] = [row['W_MOV'] if row['W_loc'] == 'H' else row['L_MOV'] for i, row in Combined_FF.iterrows()]

In [47]:
Combined_FF.head()

,WTeamID,LTeamID,W_loc,WScore,LScore,W_Name,W_Seed,W_Wins,W_Losses,W_WIN,...,H_OeFG%,H_OTOV%,H_ORB,H_DeFG%,H_DTOV%,H_DRB,H_FTP,H_PPG,H_OPPG,H_MOV
0,1104,1272,N,82,70,Alabama,9,19,15,0.56,...,0.51,0.18,0.35,0.51,0.16,0.66,0.69,70.8,71.2,-0.4
1,1107,1233,H,69,67,Albany NY,NA,21,10,0.68,...,0.50,0.15,0.35,0.51,0.15,0.76,0.75,73.3,67.9,5.4
2,1112,1319,H,101,67,Arizona,4,27,7,0.79,...,0.57,0.16,0.33,0.49,0.15,0.75,0.76,80.9,71.2,9.7
3,1113,1226,H,94,74,Arizona St,11,20,11,0.65,...,0.54,0.13,0.28,0.51,0.18,0.69,0.74,83.5,75.3,8.2
4,1116,1359,H,95,56,Arkansas,7,23,11,0.68,...,0.54,0.13,0.29,0.50,0.17,0.68,0.68,81.1,75.5,5.6


In [48]:
Combined_FF['Away_Team'] = [row['WTeamID'] if row['W_loc'] == 'A' else row['LTeamID'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_Rank'] = [row['W_Rank'] if row['W_loc'] == 'A' else row['L_Rank'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_Score'] = [row['WScore'] if row['W_loc'] == 'A' else row['LScore'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_Name'] = [row['W_Name'] if row['W_loc'] == 'A' else row['L_Name'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_Seed'] = [row['W_Seed'] if row['W_loc'] == 'A' else row['L_Seed'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_Wins'] = [row['W_Wins'] if row['W_loc'] == 'A' else row['L_Wins'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_Losses'] = [row['W_Losses'] if row['W_loc'] == 'A' else row['L_Losses'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_WIN%'] = [row['W_WIN'] if row['W_loc'] == 'A' else row['L_WIN'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_OeFG%'] = [row['W_OeFG%'] if row['W_loc'] == 'A' else row['L_OeFG%'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_OTOV%'] = [row['W_OTOV%'] if row['W_loc'] == 'A' else row['L_OTOV%'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_ORB'] = [row['W_ORB'] if row['W_loc'] == 'A' else row['L_ORB'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_DeFG%'] = [row['W_DeFG%'] if row['W_loc'] == 'A' else row['L_DeFG%'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_DTOV%'] = [row['W_DTOV%'] if row['W_loc'] == 'A' else row['L_DTOV%'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_DRB'] = [row['W_DRB'] if row['W_loc'] == 'A' else row['L_DRB'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_FTP'] = [row['W_FTP'] if row['W_loc'] == 'A' else row['L_FTP'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_PPG'] = [row['W_PPG'] if row['W_loc'] == 'A' else row['L_PPG'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_OPPG'] = [row['W_OPPG'] if row['W_loc'] == 'A' else row['L_OPPG'] for i, row in Combined_FF.iterrows()]
Combined_FF['A_MOV'] = [row['W_MOV'] if row['W_loc'] == 'A' else row['L_MOV'] for i, row in Combined_FF.iterrows()]

In [49]:
Combined_FF.head()

,WTeamID,LTeamID,W_loc,WScore,LScore,W_Name,W_Seed,W_Wins,W_Losses,W_WIN,...,A_OeFG%,A_OTOV%,A_ORB,A_DeFG%,A_DTOV%,A_DRB,A_FTP,A_PPG,A_OPPG,A_MOV
0,1104,1272,N,82,70,Alabama,9,19,15,0.56,...,0.51,0.18,0.35,0.51,0.16,0.66,0.69,70.8,71.2,-0.4
1,1107,1233,H,69,67,Albany NY,NA,21,10,0.68,...,0.54,0.14,0.24,0.52,0.16,0.70,0.74,79.8,76.2,3.6
2,1112,1319,H,101,67,Arizona,4,27,7,0.79,...,0.45,0.19,0.27,0.55,0.17,0.70,0.70,65.0,78.7,-13.7
3,1113,1226,H,94,74,Arizona St,11,20,11,0.65,...,0.55,0.17,0.20,0.53,0.15,0.72,0.68,73.1,75.1,-2.0
4,1116,1359,H,95,56,Arkansas,7,23,11,0.68,...,0.54,0.17,0.25,0.58,0.16,0.65,0.77,76.2,83.9,-7.7


In [50]:
Combined_FFHA = Combined_FF[Combined_FF.W_loc != 'N']
Combined_FFHA.shape

(4787, 82)

In [51]:
Combined_FFHA[['H_Score','A_Score']].head()
#Combined_FFHA.drop(Combined_FFHA.iloc[:,0:36])

,H_Score,A_Score
1,69,67
2,101,67
3,94,74
4,95,56
5,102,74


In [52]:
#FOUR FACTORS
Combined_FFHA['N_eFG%'] = (((Combined_FFHA['H_OeFG%'] - Combined_FFHA['A_OeFG%'])*100)-((Combined_FFHA['H_DeFG%'] - Combined_FFHA['A_DeFG%'])*100)).round(2)#*.4
Combined_FFHA['N_OeFG%'] = (((Combined_FFHA['H_OeFG%'] - Combined_FFHA['A_OeFG%'])*100)).round(2)
Combined_FFHA['N_DeFG%'] = (((Combined_FFHA['H_DeFG%'] - Combined_FFHA['A_DeFG%'])*100)).round(2)
Combined_FFHA['N_TOV%'] = (((Combined_FFHA['H_OTOV%'] - Combined_FFHA['A_OTOV%'])*100)+((Combined_FFHA['H_DTOV%'] - Combined_FFHA['A_DTOV%'])*100)).round(2)#*.25
Combined_FFHA['N_OTOV%'] = (((Combined_FFHA['H_OTOV%'] - Combined_FFHA['A_OTOV%'])*100)).round(2)
Combined_FFHA['N_DTOV%'] = (((Combined_FFHA['H_DTOV%'] - Combined_FFHA['A_DTOV%'])*100)).round(2)
Combined_FFHA['N_RB'] = (((Combined_FFHA['H_ORB'] - Combined_FFHA['A_ORB'])*100)-((Combined_FFHA['H_DRB'] - Combined_FFHA['A_DRB'])*100)).round(2)#*.25
Combined_FFHA['N_ORB'] = (((Combined_FFHA['H_ORB'] - Combined_FFHA['A_ORB'])*100)).round(2)
Combined_FFHA['N_DRB'] = (((Combined_FFHA['H_DRB'] - Combined_FFHA['A_DRB'])*100)).round(2)
Combined_FFHA['N_FTP'] = (Combined_FFHA['H_FTP'] - Combined_FFHA['A_FTP']).round(2)

C:\Users\fjmurias.DGS\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\fjmurias.DGS\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\fjmurias.DGS\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [53]:
#OTHER FACTORS
Combined_FFHA['N_Rank'] = (((Combined_FFHA['H_Rank'] - Combined_FFHA['A_Rank']))).round(2)
Combined_FFHA['N_WIN%'] = Combined_FFHA['H_WIN%'] - Combined_FFHA['A_WIN%']
Combined_FFHA['N_PPG'] = Combined_FFHA['H_PPG'] - Combined_FFHA['A_PPG']
Combined_FFHA['N_OPPG'] = Combined_FFHA['H_OPPG'] - Combined_FFHA['A_OPPG']
Combined_FFHA['N_MOV'] = Combined_FFHA['H_MOV'] - Combined_FFHA['A_MOV']
# come back and do seed here. for non ranked teams make it a big number like 30

C:\Users\fjmurias.DGS\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\fjmurias.DGS\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\fjmurias.DGS\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [54]:
Combined_FFHA[['Home_Team','Away_Team','H_Rank','A_Rank','N_Rank','H_PPG','A_PPG','H_OPPG','A_OPPG']].head()

,Home_Team,Away_Team,H_Rank,A_Rank,N_Rank,H_PPG,A_PPG,H_OPPG,A_OPPG
1,1107,1233,152,134,18,73.3,79.8,67.9,76.2
2,1112,1319,21,326,-305,80.9,65.0,71.2,78.7
3,1113,1226,45,221,-176,83.5,73.1,75.3,75.1
4,1116,1359,37,297,-260,81.1,76.2,75.5,83.9
5,1120,1313,16,313,-297,83.4,71.3,73.3,74.1


In [55]:
Combined_FFHA.head()

,WTeamID,LTeamID,W_loc,WScore,LScore,W_Name,W_Seed,W_Wins,W_Losses,W_WIN,...,N_DTOV%,N_RB,N_ORB,N_DRB,N_FTP,N_Rank,N_WIN%,N_PPG,N_OPPG,N_MOV
1,1107,1233,H,69,67,Albany NY,NA,21,10,0.68,...,-1.0,5.0,11.0,6.0,0.01,18,0.07,-6.5,-8.3,1.8
2,1112,1319,H,101,67,Arizona,4,27,7,0.79,...,-2.0,1.0,6.0,5.0,0.06,-305,0.66,15.9,-7.5,23.4
3,1113,1226,H,94,74,Arizona St,11,20,11,0.65,...,3.0,11.0,8.0,-3.0,0.06,-176,0.22,10.4,0.2,10.2
4,1116,1359,H,95,56,Arkansas,7,23,11,0.68,...,1.0,1.0,4.0,3.0,-0.09,-260,0.41,4.9,-8.4,13.3
5,1120,1313,H,102,74,Auburn,4,25,7,0.78,...,0.0,-1.0,1.0,2.0,0.12,-297,0.35,12.1,-0.8,12.9


In [56]:
Combined_FFHA = Combined_FFHA.reindex_axis(['W_loc','Home_Team','Away_Team','H_Name','A_Name','H_Rank','A_Rank','H_Score','A_Score','H_Seed','A_Seed','N_Rank','N_WIN%','N_OeFG%','N_DeFG%','N_OTOV%','N_DTOV%','N_ORB','N_DRB','N_FTP','N_PPG','N_OPPG','N_MOV'],axis=1)
Combined_FFHA.head()

,W_loc,Home_Team,Away_Team,H_Name,A_Name,H_Rank,A_Rank,H_Score,A_Score,H_Seed,...,N_OeFG%,N_DeFG%,N_OTOV%,N_DTOV%,N_ORB,N_DRB,N_FTP,N_PPG,N_OPPG,N_MOV
1,H,1107,1233,Albany NY,Iona,152,134,69,67,NA,...,-4.0,-1.0,1.0,-1.0,11.0,6.0,0.01,-6.5,-8.3,1.8
2,H,1112,1319,Arizona,Northern Arizona,21,326,101,67,4,...,12.0,-6.0,-3.0,-2.0,6.0,5.0,0.06,15.9,-7.5,23.4
3,H,1113,1226,Arizona St,Idaho St,45,221,94,74,11,...,-1.0,-2.0,-4.0,3.0,8.0,-3.0,0.06,10.4,0.2,10.2
4,H,1116,1359,Arkansas,Samford,37,297,95,56,7,...,0.0,-8.0,-4.0,1.0,4.0,3.0,-0.09,4.9,-8.4,13.3
5,H,1120,1313,Auburn,Norfolk St,16,313,102,74,4,...,3.0,-1.0,-4.0,0.0,1.0,2.0,0.12,12.1,-0.8,12.9


In [57]:
Home_W = []
for team in Combined_FFHA['W_loc']:
    if team == 'H':
        Home_W.append(1)
    else:
        Home_W.append(0)

In [58]:
Combined_FFHA['Home_W'] = Home_W

In [59]:
Combined_FFHA.head()

,W_loc,Home_Team,Away_Team,H_Name,A_Name,H_Rank,A_Rank,H_Score,A_Score,H_Seed,...,N_DeFG%,N_OTOV%,N_DTOV%,N_ORB,N_DRB,N_FTP,N_PPG,N_OPPG,N_MOV,Home_W
1,H,1107,1233,Albany NY,Iona,152,134,69,67,NA,...,-1.0,1.0,-1.0,11.0,6.0,0.01,-6.5,-8.3,1.8,1
2,H,1112,1319,Arizona,Northern Arizona,21,326,101,67,4,...,-6.0,-3.0,-2.0,6.0,5.0,0.06,15.9,-7.5,23.4,1
3,H,1113,1226,Arizona St,Idaho St,45,221,94,74,11,...,-2.0,-4.0,3.0,8.0,-3.0,0.06,10.4,0.2,10.2,1
4,H,1116,1359,Arkansas,Samford,37,297,95,56,7,...,-8.0,-4.0,1.0,4.0,3.0,-0.09,4.9,-8.4,13.3,1
5,H,1120,1313,Auburn,Norfolk St,16,313,102,74,4,...,-1.0,-4.0,0.0,1.0,2.0,0.12,12.1,-0.8,12.9,1


In [60]:
Feature_cols = ['N_Rank','N_OeFG%','N_DeFG%','N_OTOV%','N_DTOV%','N_ORB','N_DRB','N_FTP','N_MOV']
X = Combined_FFHA[Feature_cols]
y = Combined_FFHA.Home_W 

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

C:\Users\fjmurias.DGS\AppData\Local\Continuum\anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [61]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [62]:
#logreg.coef_

In [63]:
pred = logreg.predict(X)

In [64]:
pred[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [65]:
y_pred_class = logreg.predict(X_test)

In [66]:
y_pred_class[:10]

array([1, 1, 1, 1, 1, 0, 0, 1, 1, 1], dtype=int64)

In [67]:
y_pred_prob = logreg.predict_proba(X)

In [68]:
y_pred_prob[0:10]

array([[ 0.41964894,  0.58035106],
       [ 0.00450738,  0.99549262],
       [ 0.03794401,  0.96205599],
       [ 0.01233243,  0.98766757],
       [ 0.00803656,  0.99196344],
       [ 0.05949583,  0.94050417],
       [ 0.28138952,  0.71861048],
       [ 0.01279078,  0.98720922],
       [ 0.37192452,  0.62807548],
       [ 0.00984662,  0.99015338]])

In [69]:
# store the predicted probabilites of class 1
Combined_FFHA['Home_W_prob'] = logreg.predict_proba(X)[:, 1]
Combined_FFHA.head(10)

,W_loc,Home_Team,Away_Team,H_Name,A_Name,H_Rank,A_Rank,H_Score,A_Score,H_Seed,...,N_OTOV%,N_DTOV%,N_ORB,N_DRB,N_FTP,N_PPG,N_OPPG,N_MOV,Home_W,Home_W_prob
1,H,1107,1233,Albany NY,Iona,152,134,69,67,NA,...,1.0,-1.0,11.0,6.0,0.01,-6.5,-8.3,1.8,1,0.580351
2,H,1112,1319,Arizona,Northern Arizona,21,326,101,67,4,...,-3.0,-2.0,6.0,5.0,0.06,15.9,-7.5,23.4,1,0.995493
3,H,1113,1226,Arizona St,Idaho St,45,221,94,74,11,...,-4.0,3.0,8.0,-3.0,0.06,10.4,0.2,10.2,1,0.962056
4,H,1116,1359,Arkansas,Samford,37,297,95,56,7,...,-4.0,1.0,4.0,3.0,-0.09,4.9,-8.4,13.3,1,0.987668
5,H,1120,1313,Auburn,Norfolk St,16,313,102,74,4,...,-4.0,0.0,1.0,2.0,0.12,12.1,-0.8,12.9,1,0.991963
6,H,1124,1146,Baylor,Cent Arkansas,34,201,107,66,NA,...,1.0,-2.0,8.0,4.0,0.00,-5.8,-10.1,4.3,1,0.940504
7,H,1127,1288,Binghamton,Morgan St,300,323,76,59,NA,...,0.0,-3.0,-8.0,4.0,-0.09,-6.3,-5.5,-0.8,1,0.718610
8,H,1130,1263,Boston College,Maine,73,333,85,65,NA,...,-3.0,0.0,5.0,4.0,0.05,11.4,-2.3,13.7,1,0.987209
9,A,1180,1132,Drexel,Bowling Green,253,238,69,78,NA,...,0.0,-1.0,1.0,-5.0,-0.02,-0.4,0.8,-1.2,0,0.628075
10,H,1139,1244,Butler,Kennesaw,25,311,82,64,10,...,-3.0,2.0,-2.0,9.0,0.08,7.4,-5.7,13.1,1,0.990153


In [128]:
Combined_FFHA.shape

(4787, 25)

In [130]:
Combined_FFHA['W_loc'].value_counts()

H    3173
A    1614
Name: W_loc, dtype: int64

In [70]:
Combined_FFHA.to_csv('Combined_FFHAKP2.csv')

In [71]:
from sklearn import metrics
metrics.accuracy_score(y_test,y_pred_class)

0.75856307435254799

In [72]:
metrics.log_loss(y_test,y_pred_class)

8.3390528886211133

In [73]:
teaminfo2018.head()

,Teams,Name,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
0,1101,Abilene Chr,NA,12,15,0.44,0.51,0.18,0.27,0.52,0.19,0.70,0.70,70.7,72.8,-2.1
1,1102,Air Force,NA,10,19,0.34,0.48,0.17,0.27,0.55,0.18,0.71,0.74,66.7,72.7,-6.0
2,1103,Akron,NA,13,18,0.42,0.52,0.18,0.27,0.54,0.17,0.72,0.70,71.3,76.2,-4.9
3,1104,Alabama,9,19,15,0.56,0.52,0.18,0.29,0.47,0.16,0.69,0.67,72.4,70.0,2.4
4,1105,Alabama A&M,NA,3,28,0.10,0.45,0.21,0.30,0.54,0.14,0.67,0.65,60.4,76.4,-16.0


In [74]:
teaminfo2018KP = pd.DataFrame()
teaminfo2018KP = pd.merge(left=teaminfo2018,right=kenpom2018, how='left', left_on='Teams', right_on='TeamID') 
teaminfo2018KP.head()

,Teams,Name,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,...,DRB,FTP,PPG,OPPG,MOV,Season,RankingDayNum,SystemName,TeamID,Rank
0,1101,Abilene Chr,NA,12,15,0.44,0.51,0.18,0.27,0.52,...,0.70,0.70,70.7,72.8,-2.1,2018,133,POM,1101,265
1,1102,Air Force,NA,10,19,0.34,0.48,0.17,0.27,0.55,...,0.71,0.74,66.7,72.7,-6.0,2018,133,POM,1102,232
2,1103,Akron,NA,13,18,0.42,0.52,0.18,0.27,0.54,...,0.72,0.70,71.3,76.2,-4.9,2018,133,POM,1103,258
3,1104,Alabama,9,19,15,0.56,0.52,0.18,0.29,0.47,...,0.69,0.67,72.4,70.0,2.4,2018,133,POM,1104,51
4,1105,Alabama A&M,NA,3,28,0.10,0.45,0.21,0.30,0.54,...,0.67,0.65,60.4,76.4,-16.0,2018,133,POM,1105,351


In [75]:
teaminfo2018KP = teaminfo2018KP.reindex_axis(['Teams','Name','Rank','Seed','Wins','Losses','WIN','OeFG%','OTOV%','ORB','DeFG%','DTOV%','DRB','FTP','PPG','OPPG','MOV'],axis=1)

In [76]:
tourneyteaminfo2018 = teaminfo2018KP[teaminfo2018KP.Seed != ' NA']

In [77]:
tourneyteaminfo2018.head()

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
3,1104,Alabama,51,9,19,15,0.56,0.52,0.18,0.29,0.47,0.16,0.69,0.67,72.4,70.0,2.4
10,1112,Arizona,21,4,27,7,0.79,0.57,0.16,0.33,0.49,0.15,0.75,0.76,80.9,71.2,9.7
11,1113,Arizona St,45,11,20,11,0.65,0.54,0.13,0.28,0.51,0.18,0.69,0.74,83.5,75.3,8.2
14,1116,Arkansas,37,7,23,11,0.68,0.54,0.13,0.29,0.50,0.17,0.68,0.68,81.1,75.5,5.6
17,1120,Auburn,16,4,25,7,0.78,0.52,0.14,0.33,0.49,0.18,0.72,0.79,83.4,73.3,10.1


In [78]:
df_tourneymatchups2018 = pd.read_csv('2018 SampleSubmission.csv')

In [79]:
df_tourneymatchups2018.head()

,ID,Pred
0,2018_1104_1112,0.5
1,2018_1104_1113,0.5
2,2018_1104_1116,0.5
3,2018_1104_1120,0.5
4,2018_1104_1137,0.5


In [80]:
def SeedSlice(seed):
    s_int = int(seed[5:9])
    return s_int
df_tourneymatchups2018['Home'] = df_tourneymatchups2018.ID.apply(SeedSlice)

In [81]:
def SeedSlice(seed):
    s_int = int(seed[10:14])
    return s_int
df_tourneymatchups2018['Away'] = df_tourneymatchups2018.ID.apply(SeedSlice)

In [82]:
#df_tourneymatchups2017.drop('Pred', axis=1, inplace=True)

In [83]:
df_tourneymatchups2018.head()

,ID,Pred,Home,Away
0,2018_1104_1112,0.5,1104,1112
1,2018_1104_1113,0.5,1104,1113
2,2018_1104_1116,0.5,1104,1116
3,2018_1104_1120,0.5,1104,1120
4,2018_1104_1137,0.5,1104,1137


In [84]:
df_tourneymatchups2018W = pd.DataFrame()
df_tourneymatchups2018W = pd.merge(left=df_tourneymatchups2018,right=tourneyteaminfo2018, how='left', left_on='Home', right_on='Teams') 
df_tourneymatchups2018W.head()

,ID,Pred,Home,Away,Teams,Name,Rank,Seed,Wins,Losses,...,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
0,2018_1104_1112,0.5,1104,1112,1104,Alabama,51,9,19,15,...,0.52,0.18,0.29,0.47,0.16,0.69,0.67,72.4,70.0,2.4
1,2018_1104_1113,0.5,1104,1113,1104,Alabama,51,9,19,15,...,0.52,0.18,0.29,0.47,0.16,0.69,0.67,72.4,70.0,2.4
2,2018_1104_1116,0.5,1104,1116,1104,Alabama,51,9,19,15,...,0.52,0.18,0.29,0.47,0.16,0.69,0.67,72.4,70.0,2.4
3,2018_1104_1120,0.5,1104,1120,1104,Alabama,51,9,19,15,...,0.52,0.18,0.29,0.47,0.16,0.69,0.67,72.4,70.0,2.4
4,2018_1104_1137,0.5,1104,1137,1104,Alabama,51,9,19,15,...,0.52,0.18,0.29,0.47,0.16,0.69,0.67,72.4,70.0,2.4


In [85]:
TourneyGames_2018 = pd.DataFrame()
TourneyGames_2018 = pd.merge(left=df_tourneymatchups2018W,right=tourneyteaminfo2018, how='left', left_on='Away', right_on='Teams') 
TourneyGames_2018.head()

,ID,Pred,Home,Away,Teams_x,Name_x,Rank_x,Seed_x,Wins_x,Losses_x,...,OeFG%_y,OTOV%_y,ORB_y,DeFG%_y,DTOV%_y,DRB_y,FTP_y,PPG_y,OPPG_y,MOV_y
0,2018_1104_1112,0.5,1104,1112,1104,Alabama,51,9,19,15,...,0.57,0.16,0.33,0.49,0.15,0.75,0.76,80.9,71.2,9.7
1,2018_1104_1113,0.5,1104,1113,1104,Alabama,51,9,19,15,...,0.54,0.13,0.28,0.51,0.18,0.69,0.74,83.5,75.3,8.2
2,2018_1104_1116,0.5,1104,1116,1104,Alabama,51,9,19,15,...,0.54,0.13,0.29,0.50,0.17,0.68,0.68,81.1,75.5,5.6
3,2018_1104_1120,0.5,1104,1120,1104,Alabama,51,9,19,15,...,0.52,0.14,0.33,0.49,0.18,0.72,0.79,83.4,73.3,10.1
4,2018_1104_1137,0.5,1104,1137,1104,Alabama,51,9,19,15,...,0.54,0.15,0.27,0.48,0.15,0.73,0.72,81.1,72.9,8.2


In [86]:
TourneyGames_2018.rename(columns={'Name_x': 'H_Name','Rank_x': 'H_Rank', 'Seed_x': 'H_Seed','Wins_x': 'H_Wins','Losses_x': 'H_Losses','WIN_x': 'H_WIN%','OeFG%_x': 'H_OeFG%','OTOV%_x': 'H_OTOV%','ORB_x': 'H_ORB','DeFG%_x': 'H_DeFG%','DTOV%_x': 'H_DTOV%','DRB_x': 'H_DRB','FTP_x': 'H_FTP','PPG_x': 'H_PPG','OPPG_x': 'H_OPPG','MOV_x': 'H_MOV'}, inplace=True)

In [87]:
TourneyGames_2018.rename(columns={'Name_y': 'A_Name','Rank_y': 'A_Rank', 'Seed_y': 'A_Seed','Wins_y': 'A_Wins','Losses_y': 'A_Losses','WIN_y': 'A_WIN%','OeFG%_y': 'A_OeFG%','OTOV%_y': 'A_OTOV%','ORB_y': 'A_ORB','DeFG%_y': 'A_DeFG%','DTOV%_y': 'A_DTOV%','DRB_y': 'A_DRB','FTP_y': 'A_FTP','PPG_y': 'A_PPG','OPPG_y': 'A_OPPG','MOV_y': 'A_MOV'}, inplace=True)

In [88]:
TourneyGames_2018['H_Rank'].head()

0    51
1    51
2    51
3    51
4    51
Name: H_Rank, dtype: int64

In [89]:
TourneyGames_2018.head()

,ID,Pred,Home,Away,Teams_x,H_Name,H_Rank,H_Seed,H_Wins,H_Losses,...,A_OeFG%,A_OTOV%,A_ORB,A_DeFG%,A_DTOV%,A_DRB,A_FTP,A_PPG,A_OPPG,A_MOV
0,2018_1104_1112,0.5,1104,1112,1104,Alabama,51,9,19,15,...,0.57,0.16,0.33,0.49,0.15,0.75,0.76,80.9,71.2,9.7
1,2018_1104_1113,0.5,1104,1113,1104,Alabama,51,9,19,15,...,0.54,0.13,0.28,0.51,0.18,0.69,0.74,83.5,75.3,8.2
2,2018_1104_1116,0.5,1104,1116,1104,Alabama,51,9,19,15,...,0.54,0.13,0.29,0.50,0.17,0.68,0.68,81.1,75.5,5.6
3,2018_1104_1120,0.5,1104,1120,1104,Alabama,51,9,19,15,...,0.52,0.14,0.33,0.49,0.18,0.72,0.79,83.4,73.3,10.1
4,2018_1104_1137,0.5,1104,1137,1104,Alabama,51,9,19,15,...,0.54,0.15,0.27,0.48,0.15,0.73,0.72,81.1,72.9,8.2


In [90]:
tourneyteaminfo2018.sort_values('Rank',ascending = True).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
326,1438,Virginia,1,1,31,2,0.94,0.53,0.12,0.27,0.44,0.19,0.74,0.76,67.5,53.4,14.1
325,1437,Villanova,2,1,30,4,0.88,0.60,0.13,0.29,0.49,0.16,0.73,0.77,87.1,70.9,16.2


In [91]:
tourneyteaminfo2018.sort_values('MOV',ascending =  False).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
104,1211,Gonzaga,8,4,30,4,0.88,0.58,0.14,0.33,0.47,0.16,0.77,0.72,84.5,67.1,17.4
46,1153,Cincinnati,4,2,30,4,0.88,0.52,0.15,0.38,0.43,0.19,0.71,0.69,74.5,57.1,17.4


In [92]:
tourneyteaminfo2018.sort_values('ORB',ascending =  False).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
74,1181,Duke,3,2,26,7,0.79,0.56,0.15,0.39,0.46,0.15,0.71,0.71,84.7,69.6,15.1
204,1314,North Carolina,7,2,25,10,0.71,0.52,0.14,0.38,0.50,0.14,0.74,0.74,82.0,73.1,8.9


In [93]:
tourneyteaminfo2018.sort_values('OPPG',ascending = False).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
217,1328,Oklahoma,47,10,18,13,0.58,0.54,0.15,0.28,0.51,0.15,0.70,0.75,85.2,81.6,3.6
300,1411,TX Southern,249,16,15,19,0.44,0.51,0.15,0.29,0.52,0.14,0.68,0.72,77.6,79.7,-2.1


In [94]:
tourneyteaminfo2018.sort_values('OPPG',ascending = True).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
326,1438,Virginia,1,1,31,2,0.94,0.53,0.12,0.27,0.44,0.19,0.74,0.76,67.5,53.4,14.1
46,1153,Cincinnati,4,2,30,4,0.88,0.52,0.15,0.38,0.43,0.19,0.71,0.69,74.5,57.1,17.4


In [95]:
tourneyteaminfo2018.sort_values('DeFG%',ascending = True).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
46,1153,Cincinnati,4,2,30,4,0.88,0.52,0.15,0.38,0.43,0.19,0.71,0.69,74.5,57.1,17.4
169,1277,Michigan St,6,3,29,4,0.88,0.58,0.17,0.36,0.43,0.13,0.73,0.75,81.0,64.8,16.2


In [96]:
tourneyteaminfo2018.sort_values('DeFG%',ascending = True).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
46,1153,Cincinnati,4,2,30,4,0.88,0.52,0.15,0.38,0.43,0.19,0.71,0.69,74.5,57.1,17.4
169,1277,Michigan St,6,3,29,4,0.88,0.58,0.17,0.36,0.43,0.13,0.73,0.75,81.0,64.8,16.2


In [97]:
tourneyteaminfo2018.sort_values('FTP',ascending = False).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
65,1172,Davidson,43,12,21,11,0.66,0.58,0.13,0.23,0.51,0.15,0.77,0.80,76.4,67.6,8.8
348,1462,Xavier,14,1,28,5,0.85,0.56,0.15,0.30,0.50,0.14,0.77,0.79,84.3,74.5,9.8


In [98]:
tourneyteaminfo2018.sort_values('OeFG%',ascending = True).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
282,1393,Syracuse,54,11,20,13,0.61,0.47,0.16,0.35,0.47,0.17,0.71,0.74,67.5,64.5,3.0
236,1347,Radford,170,16,20,12,0.62,0.49,0.16,0.32,0.49,0.18,0.72,0.73,66.9,64.9,2.0


In [99]:
tourneyteaminfo2018.sort_values('DeFG%',ascending = True).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
46,1153,Cincinnati,4,2,30,4,0.88,0.52,0.15,0.38,0.43,0.19,0.71,0.69,74.5,57.1,17.4
169,1277,Michigan St,6,3,29,4,0.88,0.58,0.17,0.36,0.43,0.13,0.73,0.75,81.0,64.8,16.2


In [100]:
tourneyteaminfo2018.sort_values('OeFG%',ascending = False).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
325,1437,Villanova,2,1,30,4,0.88,0.60,0.13,0.29,0.49,0.16,0.73,0.77,87.1,70.9,16.2
134,1242,Kansas,9,1,27,7,0.79,0.58,0.15,0.29,0.48,0.16,0.68,0.70,81.5,70.9,10.6


In [101]:
tourneyteaminfo2018.sort_values('WIN',ascending = False).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
326,1438,Virginia,1,1,31,2,0.94,0.53,0.12,0.27,0.44,0.19,0.74,0.76,67.5,53.4,14.1
325,1437,Villanova,2,1,30,4,0.88,0.60,0.13,0.29,0.49,0.16,0.73,0.77,87.1,70.9,16.2


In [102]:
tourneyteaminfo2018.sort_values('WIN',ascending = True).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
300,1411,TX Southern,249,16,15,19,0.44,0.51,0.15,0.29,0.52,0.14,0.68,0.72,77.6,79.7,-2.1
146,1254,Long Island,251,16,17,16,0.52,0.52,0.17,0.30,0.50,0.14,0.70,0.70,76.7,76.8,-0.1


In [103]:
tourneyteaminfo2018.sort_values('ORB',ascending = True).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
159,1267,Marshall,114,13,23,10,0.70,0.55,0.16,0.22,0.50,0.16,0.68,0.77,83.6,79.1,4.5
152,1260,Loyola-Chicago,41,11,27,5,0.84,0.58,0.17,0.22,0.47,0.18,0.74,0.72,71.7,62.0,9.7


In [104]:
tourneyteaminfo2018.sort_values('ORB',ascending = False).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
74,1181,Duke,3,2,26,7,0.79,0.56,0.15,0.39,0.46,0.15,0.71,0.71,84.7,69.6,15.1
204,1314,North Carolina,7,2,25,10,0.71,0.52,0.14,0.38,0.50,0.14,0.74,0.74,82.0,73.1,8.9


In [105]:
tourneyteaminfo2018.sort_values('DRB',ascending = False).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
348,1462,Xavier,14,1,28,5,0.85,0.56,0.15,0.30,0.5,0.14,0.77,0.79,84.3,74.5,9.8
244,1355,S Dakota St,75,12,24,6,0.80,0.54,0.12,0.25,0.5,0.13,0.77,0.76,83.4,75.6,7.8


In [106]:
tourneyteaminfo2018.sort_values('OTOV%',ascending = True).head(2)

,Teams,Name,Rank,Seed,Wins,Losses,WIN,OeFG%,OTOV%,ORB,DeFG%,DTOV%,DRB,FTP,PPG,OPPG,MOV
195,1305,Nevada,24,7,27,7,0.79,0.55,0.12,0.27,0.49,0.16,0.71,0.75,83.1,72.9,10.2
168,1276,Michigan,10,3,27,7,0.79,0.54,0.12,0.26,0.48,0.17,0.76,0.66,73.6,63.5,10.1


In [107]:
#TourneyGames_2018.head()

In [108]:
#FOUR FACTORS
TourneyGames_2018['N_eFG%'] = (((TourneyGames_2018['H_OeFG%'] - TourneyGames_2018['A_OeFG%'])*100)-((TourneyGames_2018['H_DeFG%'] - TourneyGames_2018['A_DeFG%'])*100)).round(2)#*.4
TourneyGames_2018['N_OeFG%'] = (((TourneyGames_2018['H_OeFG%'] - TourneyGames_2018['A_OeFG%'])*100)).round(2)
TourneyGames_2018['N_DeFG%'] = (((TourneyGames_2018['H_DeFG%'] - TourneyGames_2018['A_DeFG%'])*100)).round(2)
TourneyGames_2018['N_TOV%'] = ((-(TourneyGames_2018['H_OTOV%'] - TourneyGames_2018['A_OTOV%'])*100)+((TourneyGames_2018['H_DTOV%'] - TourneyGames_2018['A_DTOV%'])*100)).round(2)#*.25
TourneyGames_2018['N_OTOV%'] = (((TourneyGames_2018['H_OTOV%'] - TourneyGames_2018['A_OTOV%'])*100)).round(2)
TourneyGames_2018['N_DTOV%'] = (((TourneyGames_2018['H_DTOV%'] - TourneyGames_2018['A_DTOV%'])*100)).round(2)
TourneyGames_2018['N_RB'] = (((TourneyGames_2018['H_ORB'] - TourneyGames_2018['A_ORB'])*100)+((TourneyGames_2018['H_DRB'] - TourneyGames_2018['A_DRB'])*100)).round(2)#*.25
TourneyGames_2018['N_ORB'] = (((TourneyGames_2018['H_ORB'] - TourneyGames_2018['A_ORB'])*100)).round(2)
TourneyGames_2018['N_DRB'] = (((TourneyGames_2018['H_DRB'] - TourneyGames_2018['A_DRB'])*100)).round(2)
TourneyGames_2018['N_FTP'] = (TourneyGames_2018['H_FTP'] - TourneyGames_2018['A_FTP']).round(2)#*.15

In [109]:
#OTHER FACTORS
TourneyGames_2018['N_Rank'] = (TourneyGames_2018['H_Rank'] - TourneyGames_2018['A_Rank'])
TourneyGames_2018['N_WIN%'] = TourneyGames_2018['H_WIN%'] - TourneyGames_2018['A_WIN%']
TourneyGames_2018['N_PPG'] = TourneyGames_2018['H_PPG'] - TourneyGames_2018['A_PPG']
TourneyGames_2018['N_OPPG'] = TourneyGames_2018['H_OPPG'] - TourneyGames_2018['A_OPPG']
TourneyGames_2018['N_MOV'] = TourneyGames_2018['H_MOV'] - TourneyGames_2018['A_MOV']
# come back and do seed here. for non ranked teams make it a big number like 30

In [110]:
TourneyGames_2018.head()

,ID,Pred,Home,Away,Teams_x,H_Name,H_Rank,H_Seed,H_Wins,H_Losses,...,N_DTOV%,N_RB,N_ORB,N_DRB,N_FTP,N_Rank,N_WIN%,N_PPG,N_OPPG,N_MOV
0,2018_1104_1112,0.5,1104,1112,1104,Alabama,51,9,19,15,...,1.0,-10.0,-4.0,-6.0,-0.09,30,-0.23,-8.5,-1.2,-7.3
1,2018_1104_1113,0.5,1104,1113,1104,Alabama,51,9,19,15,...,-2.0,1.0,1.0,0.0,-0.07,6,-0.09,-11.1,-5.3,-5.8
2,2018_1104_1116,0.5,1104,1116,1104,Alabama,51,9,19,15,...,-1.0,1.0,0.0,1.0,-0.01,14,-0.12,-8.7,-5.5,-3.2
3,2018_1104_1120,0.5,1104,1120,1104,Alabama,51,9,19,15,...,-2.0,-7.0,-4.0,-3.0,-0.12,35,-0.22,-11.0,-3.3,-7.7
4,2018_1104_1137,0.5,1104,1137,1104,Alabama,51,9,19,15,...,1.0,-2.0,2.0,-4.0,-0.05,-49,-0.18,-8.7,-2.9,-5.8


In [111]:
TourneyGames_2018 = TourneyGames_2018.reindex_axis(['Home','Away','H_Name','A_Name','H_Seed','A_Seed','N_Rank','N_WIN%','N_OeFG%','N_DeFG%','N_OTOV%','N_DTOV%','N_ORB','N_DRB','N_FTP','N_PPG','N_OPPG','N_MOV'],axis=1)

In [112]:
TourneyGames_2018.head()

,Home,Away,H_Name,A_Name,H_Seed,A_Seed,N_Rank,N_WIN%,N_OeFG%,N_DeFG%,N_OTOV%,N_DTOV%,N_ORB,N_DRB,N_FTP,N_PPG,N_OPPG,N_MOV
0,1104,1112,Alabama,Arizona,9,4,30,-0.23,-5.0,-2.0,2.0,1.0,-4.0,-6.0,-0.09,-8.5,-1.2,-7.3
1,1104,1113,Alabama,Arizona St,9,11,6,-0.09,-2.0,-4.0,5.0,-2.0,1.0,0.0,-0.07,-11.1,-5.3,-5.8
2,1104,1116,Alabama,Arkansas,9,7,14,-0.12,-2.0,-3.0,5.0,-1.0,0.0,1.0,-0.01,-8.7,-5.5,-3.2
3,1104,1120,Alabama,Auburn,9,4,35,-0.22,0.0,-2.0,4.0,-2.0,-4.0,-3.0,-0.12,-11.0,-3.3,-7.7
4,1104,1137,Alabama,Bucknell,9,14,-49,-0.18,-2.0,-1.0,3.0,1.0,2.0,-4.0,-0.05,-8.7,-2.9,-5.8


In [113]:
TourneyGames_2018['N_Seed'] = TourneyGames_2018['A_Seed'] - TourneyGames_2018['H_Seed'] 

In [114]:
TourneyGames_2018.tail()

,Home,Away,H_Name,A_Name,H_Seed,A_Seed,N_Rank,N_WIN%,N_OeFG%,N_DeFG%,N_OTOV%,N_DTOV%,N_ORB,N_DRB,N_FTP,N_PPG,N_OPPG,N_MOV,N_Seed
2273,1452,1460,West Virginia,Wright St,5,14,-122,-0.01,1.0,2.0,-3.0,2.0,8.0,-5.0,0.05,8.3,2.8,5.5,9
2274,1452,1462,West Virginia,Xavier,5,1,-1,-0.14,-6.0,0.0,-2.0,7.0,7.0,-7.0,-0.02,-4.7,-5.5,0.8,-4
2275,1455,1460,Wichita St,Wright St,4,14,-115,0.06,6.0,1.0,-2.0,-4.0,6.0,2.0,0.02,11.7,5.1,6.6,10
2276,1455,1462,Wichita St,Xavier,4,1,6,-0.07,-1.0,-1.0,-1.0,1.0,5.0,0.0,-0.05,-1.3,-3.2,1.9,-3
2277,1460,1462,Wright St,Xavier,14,1,121,-0.13,-7.0,-2.0,1.0,5.0,-1.0,-2.0,-0.07,-13.0,-8.3,-4.7,-13


In [115]:
Home_W = []
for seed in TourneyGames_2018['N_Seed']:
    if seed > 0:
        Home_W.append(1)
    else:
        Home_W.append(0)

In [116]:
TourneyGames_2018['Home_W'] = Home_W
TourneyGames_2018.tail()

,Home,Away,H_Name,A_Name,H_Seed,A_Seed,N_Rank,N_WIN%,N_OeFG%,N_DeFG%,N_OTOV%,N_DTOV%,N_ORB,N_DRB,N_FTP,N_PPG,N_OPPG,N_MOV,N_Seed,Home_W
2273,1452,1460,West Virginia,Wright St,5,14,-122,-0.01,1.0,2.0,-3.0,2.0,8.0,-5.0,0.05,8.3,2.8,5.5,9,1
2274,1452,1462,West Virginia,Xavier,5,1,-1,-0.14,-6.0,0.0,-2.0,7.0,7.0,-7.0,-0.02,-4.7,-5.5,0.8,-4,0
2275,1455,1460,Wichita St,Wright St,4,14,-115,0.06,6.0,1.0,-2.0,-4.0,6.0,2.0,0.02,11.7,5.1,6.6,10,1
2276,1455,1462,Wichita St,Xavier,4,1,6,-0.07,-1.0,-1.0,-1.0,1.0,5.0,0.0,-0.05,-1.3,-3.2,1.9,-3,0
2277,1460,1462,Wright St,Xavier,14,1,121,-0.13,-7.0,-2.0,1.0,5.0,-1.0,-2.0,-0.07,-13.0,-8.3,-4.7,-13,0


In [117]:
Feature_cols = ['N_Rank','N_OeFG%','N_DeFG%','N_OTOV%','N_DTOV%','N_ORB','N_DRB','N_FTP','N_MOV'] #,'N_MOV' 
#X = Combined_FFHA[Feature_cols]
#y = Combined_FFHA.Home_W 

In [118]:
from sklearn.cross_validation import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train = Combined_FFHA[Feature_cols]
X_test = TourneyGames_2018[Feature_cols]
y_train = Combined_FFHA.Home_W
y_test = TourneyGames_2018.Home_W

In [119]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [120]:
pred = logreg.predict(X_train)
pred[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [121]:
y_pred_class = logreg.predict(X_test)
y_pred_class[:10]

array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1], dtype=int64)

In [122]:
y_pred_prob = logreg.predict_proba(X_test)
y_pred_prob[0:10]

array([[ 0.49560488,  0.50439512],
       [ 0.41937249,  0.58062751],
       [ 0.42937267,  0.57062733],
       [ 0.50756543,  0.49243457],
       [ 0.26163131,  0.73836869],
       [ 0.33559392,  0.66440608],
       [ 0.44830287,  0.55169713],
       [ 0.61481605,  0.38518395],
       [ 0.4699767 ,  0.5300233 ],
       [ 0.20650786,  0.79349214]])

In [123]:
# store the predicted probabilites of class 1
TourneyGames_2018['Home_W_prob'] = logreg.predict_proba(X_test)[:, 1]
TourneyGames_2018.tail(10)

,Home,Away,H_Name,A_Name,H_Seed,A_Seed,N_Rank,N_WIN%,N_OeFG%,N_DeFG%,...,N_DTOV%,N_ORB,N_DRB,N_FTP,N_PPG,N_OPPG,N_MOV,N_Seed,Home_W,Home_W_prob
2268,1439,1452,Virginia Tech,West Virginia,8,5,19,-0.05,7.0,0.0,...,-5.0,-13.0,1.0,-0.06,0.1,2.8,-2.7,-3,0,0.599324
2269,1439,1455,Virginia Tech,Wichita St,8,4,12,-0.12,2.0,1.0,...,1.0,-11.0,-6.0,-0.03,-3.3,0.5,-3.8,-4,0,0.610672
2270,1439,1460,Virginia Tech,Wright St,8,14,-103,-0.06,8.0,2.0,...,-3.0,-5.0,-4.0,-0.01,8.4,5.6,2.8,6,1,0.894378
2271,1439,1462,Virginia Tech,Xavier,8,1,18,-0.19,1.0,0.0,...,2.0,-6.0,-6.0,-0.08,-4.6,-2.7,-1.9,-7,0,0.614407
2272,1452,1455,West Virginia,Wichita St,5,4,-7,-0.07,-5.0,1.0,...,6.0,2.0,-7.0,0.03,-3.4,-2.3,-1.1,-1,0,0.671931
2273,1452,1460,West Virginia,Wright St,5,14,-122,-0.01,1.0,2.0,...,2.0,8.0,-5.0,0.05,8.3,2.8,5.5,9,1,0.917060
2274,1452,1462,West Virginia,Xavier,5,1,-1,-0.14,-6.0,0.0,...,7.0,7.0,-7.0,-0.02,-4.7,-5.5,0.8,-4,0,0.675390
2275,1455,1460,Wichita St,Wright St,4,14,-115,0.06,6.0,1.0,...,-4.0,6.0,2.0,0.02,11.7,5.1,6.6,10,1,0.913376
2276,1455,1462,Wichita St,Xavier,4,1,6,-0.07,-1.0,-1.0,...,1.0,5.0,0.0,-0.05,-1.3,-3.2,1.9,-3,0,0.664893
2277,1460,1462,Wright St,Xavier,14,1,121,-0.13,-7.0,-2.0,...,5.0,-1.0,-2.0,-0.07,-13.0,-8.3,-4.7,-13,0,0.268754


In [124]:
from sklearn import metrics
metrics.accuracy_score(y_test,y_pred_class)
# this means my model predicts the higher seed winning 69.1% of the time

0.69139596136962245

In [125]:
metrics.log_loss(y_test,y_pred_class)

10.659051941800167

In [126]:
TourneyGames_2018.to_csv('TourneyGames_2018_vKP2.csv')

In [127]:
tourneyteaminfo2018.to_csv('Tourneyteaminfo2018_vKP2.csv')

In [113]:
TourneyGames_2018.head()
TourneyGames_2018.shape

(2278, 17)

In [ ]:
#file:///C:/Users/fjmurias.DGS/Downloads/2017-mens-bracket.pdf

In [ ]:
#file:///C:/Users/fjmurias.DGS/Downloads/2017-mens-bracket.pdf